In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS
from IPython.core.display import HTML

1. Start by scraping the data from the summary page for Tennessee's 2nd District, which is available at https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N.
    * The data that we want is contained in the "Total Raised and Spent" table.
    * Make a DataFrame showing, for each candidate:
        * the candidate's name
        * the candidate's party
        * state
        * district number
        * whether the candidate was an incumbent
        * whether the candidate won the race
        * the total amount raised by that candidate (as a numeric variable)
        * the total amount spent by the candidate (as a numeric variable)

In [2]:
URL = 'https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N.'

tn2nd = requests.get(URL)

In [6]:
tn2ndsoup = BS(tn2nd.text)

In [ ]:
print(tn2ndsoup.prettify())

In [8]:
tn2ndsoup.find('title')

<title>Tennessee District 02 2020 Race • OpenSecrets</title>

In [23]:
tn2nd_table = soup.find('table', attrs={'class' : 'DataTable'})

In [ ]:
print(tn2nd_table)

In [34]:
tn2nd_list = pd.read_html(str(tn2ndsoup.find('table', attrs={'class' : 'DataTable'})))

In [ ]:
https://www.opensecrets.org/races/summary?cycle=2020&id=TN01&spec=N
https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N
https://www.opensecrets.org/races/summary?cycle=2020&id=TN03&spec=N


In [35]:
tn2nd_list

[                                Candidate      Raised     Spent Cash on Hand  \
 0  Tim Burchett (R)  • Incumbent • Winner  $1,336,276  $878,488     $593,678   
 1                         Renee Hoyos (D)    $812,784  $816,793         $210   
 
   Last Report  
 0  12/31/2020  
 1  12/31/2020  ]

In [36]:
tn2nd_df = pd.DataFrame(tn2nd_list, columns = ['Candidate', 'Party', 'State', 'District #', 'Incumbent', 'Winner', 'Raised', 'Spent'])

ValueError: Must pass 2-d input. shape=(1, 2, 5)